In [1]:
#THIS CODE DO NOT RENDER ANY PIE CHARTS

# Import required libraries
import pandas as pd
import dash
from dash import html, dcc
from dash.dependencies import Input, Output
import plotly.express as px

#this is the original code
#spacex_df = pd.read_csv("spacex_launch_dash.csv")

#next line of code added
spacex_df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Get unique launch sites
launch_sites = spacex_df['Launch Site'].unique()

# Create a dash application
app = dash.Dash(__name__)

# Function to create dropdown options
def create_dropdown_options():
    return [{'label': 'All Sites', 'value': 'ALL'}] + [{'label': site, 'value': site} for site in launch_sites]

# Function to create app layout
def create_app_layout():
    return html.Div(children=[
        html.H1('SpaceX Launch Records Dashboard',
                style={'textAlign': 'center', 'color': '#503D36',
                       'font-size': 40}),

        dcc.Dropdown(id='site-dropdown',
                     options=create_dropdown_options(),
                     value='ALL',
                     multi=False,
                     style={'width': '80%', 'margin-left': '10%'}),

        html.Div(dcc.Graph(id='success-pie-chart')),
        html.Br(),

        html.P("Payload range (Kg):"),

        dcc.RangeSlider(id='payload-slider',
                        min=min_payload,
                        max=max_payload,
                        step=100,
                        marks={min_payload: f'{min_payload} kg', max_payload: f'{max_payload} kg'},
                        value=[min_payload, max_payload]),

        html.Div(dcc.Graph(id='success-payload-scatter-chart')),
    ])

# Set the app layout
app.layout = create_app_layout()

# Callback to update the pie chart and scatter chart based on Launch Site selection and payload range
@app.callback([Output('success-pie-chart', 'figure'),
               Output('success-payload-scatter-chart', 'figure')],
              [Input('site-dropdown', 'value'),
               Input('payload-slider', 'value')])
def update_charts(selected_site, payload_range):
    # Filter data based on launch site selection and payload range
    filtered_data = spacex_df[(spacex_df['Launch Site'] == selected_site) | (selected_site == 'ALL')]
    filtered_data = filtered_data[(filtered_data['Payload Mass (kg)'] >= payload_range[0]) & (filtered_data['Payload Mass (kg)'] <= payload_range[1])]

    # Calculate success_counts and failed_counts based on the selected launch site
    success_counts = filtered_data[filtered_data['class'] == 'Success'].shape[0]
    failed_counts = filtered_data[filtered_data['class'] == 'Failure'].shape[0]

    # Update the pie chart accordingly
    pie_chart = px.pie(names=['Success', 'Failure'], values=[success_counts, failed_counts],
                      title=f'Successful Launches for {selected_site}' if selected_site != 'ALL' else 'Total Successful Launches')

    # Update the scatter chart based on payload correlation
    scatter_chart = px.scatter(filtered_data, x='Payload Mass (kg)', y='class',
                               color='class',
                               labels={'class': 'Launch Outcome'},
                               title='Correlation between Payload and Launch Success')

    return pie_chart, scatter_chart

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)
